In [ ]:
import scri
import matplotlib.pyplot as plt
from pathlib import Path
from spherical_functions import LM_index as lm
import numpy as np
from copy import deepcopy
import h5py 
import json
import pickle

plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (12,10)

In [ ]:
data_files = {}
data_files["Lev5_big_gaussian_Lev5"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian/GW_data/rh_FiniteRadii_CodeUnits/rh_FiniteRadii_CodeUnits.h5")
# data_files["Lev5_big_gaussian_constra_Lev5"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_constra/GW_data/rh_FiniteRadii_CodeUnits/rh_FiniteRadii_CodeUnits.h5")
# data_files["Lev5_big_gaussian_constra_200_Lev5"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_constra_200/GW_data/rh_FiniteRadii_CodeUnits/rh_FiniteRadii_CodeUnits.h5")
# data_files["Lev5_big_gaussian_ah_tol100_Lev5"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_ah_tol100/GW_data/rh_FiniteRadii_CodeUnits/rh_FiniteRadii_CodeUnits.h5")
# data_files["L35_GW_data_Lev3"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_Lev3/rh_FiniteRadii_CodeUnits/rh_FiniteRadii_CodeUnits.h5")
# data_files["L35_GW_data_Lev4"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_Lev4/rh_FiniteRadii_CodeUnits/rh_FiniteRadii_CodeUnits.h5")
data_files["L35_GW_data_Lev5"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_Lev5/rh_FiniteRadii_CodeUnits/rh_FiniteRadii_CodeUnits.h5")
# data_files["L35_master_GW_data_Lev3"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_Lev3/rh_FiniteRadii_CodeUnits/rh_FiniteRadii_CodeUnits.h5")
# data_files["L35_master_GW_data_Lev4"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_Lev4/rh_FiniteRadii_CodeUnits/rh_FiniteRadii_CodeUnits.h5")
data_files["L35_master_GW_data_Lev5"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_Lev5/rh_FiniteRadii_CodeUnits/rh_FiniteRadii_CodeUnits.h5")

In [ ]:
def load_shifted_data(file_path:Path):
  with h5py.File(file_path,'r') as hf:
    radius_list = []
    hf.visit(radius_list.append)

  radius_set = set()
  for radius in radius_list:
    if 'Version' in radius:
      continue
    radius_set.add(radius[1:5])

  data_dict = {}
  for radius in radius_set:
    data_dict[radius] = scri.SpEC.read_from_h5(
                            str(file_path)+f"/R{radius}.dir",
                            dataType = scri.h,
                            frameType = scri.Inertial,
                            r_is_scaled_out = True,
                            m_is_scaled_out = True,
                        )
    # print(f"Read {radius}")

  # Subtract the extraction time
  for radius in radius_set:
    data_dict[radius].t = data_dict[radius].t - float(radius)

  return data_dict

def extract_radii(file_path:Path):
  with h5py.File(file_path,'r') as hf:
    radius_list = []
    hf.visit(radius_list.append)
  return radius_list

data_dict = {}
for key in data_files:
  data_dict[key] = load_shifted_data(data_files[key])



In [ ]:
l, m = 8,0

t_min = 1000
t_min = 2500
t_max = 40000
t_max = 7500

radius_to_plot = ['0257']

for run in data_dict:
  if radius_to_plot is None:
    radius_to_plot = list(extract_radii(data_files[key]))
    radius_to_plot.sort()
    radius_to_plot = ['0257', '0265', '0274', '0283', '0293', '0304']

  for radius in radius_to_plot:
    data = data_dict[run][radius]
    min_filter = data.t > t_min
    max_filter = data.t < t_max
    filter = max_filter&min_filter
    plt.plot(data.t[filter],(data.data[:, lm(l,m,data.ell_min)][filter]),label=run+"_"+radius)
    # plt.plot(data.t[filter],np.abs(data.data[:, lm(l,m,data.ell_min)][filter]),label=radius)
# plt.yscale('log')
# plt.title(data_file.parent.parent)
plt.legend()

In [ ]:
l, m = 2, 0

t_min = 1000
t_min = 2500
t_max = 40000
t_max = 7000
# t_max = 7500

radius_to_plot = ['0257', '0265', '0274', '0283', '0293', '0304']
radius_to_plot = ['0304']

base_radius = '0304'
base_diff_run = "L35_master_GW_data_Lev5"
# base_diff_run = "L35_GW_data_Lev5"
# base_diff_run = "Lev5_big_gaussian_constra_Lev5"
# base_diff_run = "Lev5_big_gaussian_Lev5"
# base_diff_run = "L35_master_GW_data_Lev3"
print(data_dict.keys())

data_diff = data_dict[base_diff_run][base_radius]
# print(data)
reference_data_t = data_diff.t
reference_data_h = data_diff.data[:, lm(l,m,data_diff.ell_min)]

for run in data_dict:
  if radius_to_plot is None:
    radius_to_plot = list(extract_radii(data_files[key]))
    radius_to_plot.sort()
    radius_to_plot = ['0257', '0265', '0274', '0283', '0293', '0304']

  for radius in radius_to_plot:

    if radius == base_radius and base_diff_run == run:
      continue

    min_filter = reference_data_t > t_min
    max_filter = reference_data_t < t_max
    data_curr = data_dict[run][radius]
    filter = max_filter&min_filter
    data_diff = data_curr.interpolate(reference_data_t).data[:, lm(l,m,data_curr.ell_min)] - reference_data_h
    print(np.max(data_diff))
    # data_diff = data_curr.data[:, lm(l,m,data_curr.ell_min)]
    # data_diff = np.abs(data_diff)
    plt.plot(reference_data_t[filter],data_diff[filter],label=run+"_"+radius)
    # plt.plot(data.t[filter],np.abs(data.data[:, lm(l,m,data.ell_min)][filter]),label=radius)
# plt.yscale('log')
plt.title(f"diff_{base_diff_run}_{base_radius}")
plt.tight_layout()
plt.legend()

In [ ]:
radius_to_plot = list(radius_set)
radius_to_plot.sort()
# radius_to_plot = ['0257', '0265', '0274', '0283', '0293', '0304']
l, m = 2, 2

t_min = 1000
t_min = 2500
t_max = 40000
t_max = 7500

for radius in radius_to_plot:
  if (int(radius)>300):
    continue
  data = data_dict[radius]
  min_filter = data.t > t_min
  max_filter = data.t < t_max
  filter = max_filter&min_filter
  plt.plot(data.t[filter],(data.data[:, lm(l,m,data.ell_min)][filter]),label=radius)
  # plt.plot(data.t[filter],np.abs(data.data[:, lm(l,m,data.ell_min)][filter]),label=radius)
# plt.yscale('log')
plt.title(data_file.parent.parent)
plt.legend()

In [ ]:
data_dict['0257'].interpolate(data_dict["0900"].t)

In [ ]:
t_min = 100
t_max = 500
min_filter = h0257.t > t_min
max_filter = h0257.t < t_max
h0257.t[min_filter&max_filter]

In [ ]:
h0257 = scri.SpEC.read_from_h5(
      str(data)+"/R0257.dir",
      dataType = scri.h,
      frameType = scri.Inertial,
      r_is_scaled_out = True,
      m_is_scaled_out = True,
)
h0900 = scri.SpEC.read_from_h5(
      str(data)+"/R0900.dir",
      dataType = scri.h,
      frameType = scri.Inertial,
      r_is_scaled_out = True,
      m_is_scaled_out = True,
)

In [ ]:
def signed_log(arr:np.ndarray):
  negavative_indices = arr < 0.0
  positive_indices = arr > 0.0
  # zeros will be zeros
  arr_logged = np.zeros_like(arr)
  arr_logged[positive_indices] = np.log10(arr[positive_indices])
  arr_logged[negavative_indices] = np.log10(-arr[negavative_indices])
  return arr_logged


In [ ]:
extrapolated = scri.extrapolate(
    InputDirectory = "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian/GW_data/rh_FiniteRadii_CodeUnits",
    OutputDirectory = "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian/GW_data/rh_FiniteRadii_CodeUnits",
    DataFile = "rh_FiniteRadii_CodeUnits.h5",
    ChMass = 1.0, # or whatever the initial system (Christodoulou) mass is.
    UseStupidNRARFormat = True,
    DifferenceFiles = '',
    PlotFormat = '',
)

In [ ]:
h_extra = extrapolated[5]
l, m = 1, 0
plt.plot(h_extra.t,h_extra.data[:, lm(l,m,h_extra.ell_min)])
# plt.yscale('log')

In [ ]:
l, m = 0, 0
plt.plot(h0257.t,h0257.data[:, lm(l,m,h0257.ell_min)])
plt.plot(h0900.t,h0900.data[:, lm(l,m,h0900.ell_min)])
# plt.plot(h.t,h.energy_flux())